This is a simple, supervised denoising model run on ImageNet. This code is essentially identical to `n2n_imagenet.ipynb` except it uses `SupervisedDenoisingGenerator` for both training and testing.

In [1]:
import numpy as np
import keras
from importlib import reload
import os

from helpers import generators, utils, noise_models
from models import unet_n2n, irfan_2020

In [3]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

utils.check_gpu_use()

GPU DETECTED ✓
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [15]:
!pip show tensorflow 

Name: tensorflow
Version: 2.13.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /Users/antrikshdhand/Documents/github/thesis-ml/ml/virt/lib/python3.9/site-packages
Requires: tensorflow-macos
Required-by: 


In [4]:
reload(generators)
reload(utils)
reload(noise_models)

<module 'helpers.noise_models' from '/Users/antrikshdhand/Documents/github/thesis-ml/ml/main/helpers/noise_models.py'>

In [5]:
np.random.seed(42)

## Initialise data

In [7]:
TRAIN_IMAGE_DIR = "../../raw_datasets/imagenet-10k"
VAL_IMAGE_DIR = "../../raw_datasets/BSDS300/images/train"
TEST_IMAGE_DIR = "../../raw_datasets/BSDS300/images/test"

NUM_EPOCHS = 1
GPU_BATCH_SIZE = 4
DATA_BATCH_SIZE = 16

In [8]:
noise_model = noise_models.gaussian_noise

In [9]:
train_gen = generators.SupervisedDenoisingGenerator(
    image_dir=TRAIN_IMAGE_DIR,
    noise_model=noise_model,
    batch_size=DATA_BATCH_SIZE,
    patch_edge_size=192,
    zero_one_normalisation=True,
    greyscale=True
)

val_gen = generators.SupervisedDenoisingGenerator(
    image_dir=VAL_IMAGE_DIR,
    noise_model=noise_model,
    batch_size=DATA_BATCH_SIZE,
    patch_edge_size=192,
    zero_one_normalisation=True,
    greyscale=True
)

test_gen = generators.SupervisedDenoisingGenerator(
    image_dir=TEST_IMAGE_DIR,
    noise_model=noise_model,
    batch_size=DATA_BATCH_SIZE,
    patch_edge_size=192,
    zero_one_normalisation=True,
    greyscale=True
)

## Initialise model

In [10]:
model = irfan_2020.get_irfan_model(input_shape=(192, 192, 1))
# model = unet_n2n.get_unet_model(input_shape=(192, 192, 1))

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5, beta_1=0.9, beta_2=0.99, epsilon=1e-8),
    loss=keras.losses.MeanSquaredError(),
    metrics=[utils.psnr],
)

model.summary()

Model: "irfan_2020"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 192, 192, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 192, 192, 64)      640       
                                                                 
 activation (Activation)     (None, 192, 192, 64)      0         
                                                                 
 batch_normalization (Batch  (None, 192, 192, 64)      256       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 96, 96, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 96, 96, 64)        3

## Train model

In [11]:
csv_logger = keras.callbacks.CSVLogger(
    filename="models/saved/supervised_denoiser_07122024/irfan/training.log",
    separator=',',
    append=False
)

backup_callback = keras.callbacks.BackupAndRestore(
    backup_dir="models/saved/supervised_denoiser_07122024/irfan/tmp"
)

In [12]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    batch_size=GPU_BATCH_SIZE,
    epochs=NUM_EPOCHS,
    verbose=2,
    callbacks=[csv_logger, backup_callback]
)

KeyboardInterrupt: 

In [ ]:
# model.save("models/saved/supervised_denoiser_07122024/irfan/50epochs.keras")
model.save("models/saved/supervised_denoiser_07122024/unet/50epochs.keras")

In [ ]:
reload(utils)

together = utils.get_psnr_and_loss_curves(history, together=True)
# together.savefig('models/saved/supervised_denoiser_07122024/irfan/img/psnr_loss.pdf')
together.savefig('models/saved/supervised_denoiser_07122024/unet/img/psnr_loss.pdf')
together.show()

## Evaluate model

In [ ]:
evals = model.evaluate(
    test_gen,
    batch_size=GPU_BATCH_SIZE,
    verbose=2
)

In [8]:
evals

NameError: name 'evals' is not defined

In [ ]:
ground_truth_fig, patches_fig = utils.test_model_on_image(
    image_path=TEST_IMAGE_DIR + "/108005.jpg", 
    model=model,
    patch_size=192, 
    zero_one_normalisation=True, 
    greyscale=True,
    stddev=30,
    patch_coords=(160, 50)
)

ground_truth_fig.savefig('models/saved/supervised_denoiser_07122024/irfan/img/ground_truth_fig_1.pdf')
patches_fig.savefig('models/saved/supervised_denoiser_07122024/irfan/img/patches_fig_1.pdf')

# ground_truth_fig.savefig('models/saved/supervised_denoiser_07122024/unet/img/ground_truth_fig_1.pdf')
# patches_fig.savefig('models/saved/supervised_denoiser_07122024/unet/img/patches_fig_1.pdf')

In [ ]:
ground_truth_fig, patches_fig = utils.test_model_on_image(
    image_path=TEST_IMAGE_DIR + "/148089.jpg", 
    model=model,
    patch_size=192, 
    zero_one_normalisation=True, 
    greyscale=True,
    stddev=30,
    patch_coords=(240, 110)
)

ground_truth_fig.savefig('models/saved/supervised_denoiser_07122024/irfan/img/ground_truth_fig_2.pdf')
patches_fig.savefig('models/saved/supervised_denoiser_07122024/irfan/img/patches_fig_2.pdf')

# ground_truth_fig.savefig('models/saved/supervised_denoiser_07122024/unet/img/ground_truth_fig_2.pdf')
# patches_fig.savefig('models/saved/supervised_denoiser_07122024/unet/img/patches_fig_2.pdf')